In [22]:
import pandas as pd
import time 

import torch

In [23]:
df = pd.read_csv('data.csv'
                )
df.head()

,Unnamed: 0,id,episode_id,number,raw_text,timestamp_in_ms,speaking_line,character_id,location_id,raw_character_text,raw_location_text,spoken_words,normalized_text,word_count
0,0,10368,35,29,"Lisa Simpson: Maggie, look. What's that?",235000,True,9,5.0,Lisa Simpson,Simpson Home,"Maggie, look. What's that?",maggie look whats that,4.0
1,1,10369,35,30,Lisa Simpson: Lee-mur. Lee-mur.,237000,True,9,5.0,Lisa Simpson,Simpson Home,Lee-mur. Lee-mur.,lee-mur lee-mur,2.0
2,2,10370,35,31,Lisa Simpson: Zee-boo. Zee-boo.,239000,True,9,5.0,Lisa Simpson,Simpson Home,Zee-boo. Zee-boo.,zee-boo zee-boo,2.0
3,3,10372,35,33,Lisa Simpson: I'm trying to teach Maggie that ...,245000,True,9,5.0,Lisa Simpson,Simpson Home,I'm trying to teach Maggie that nature doesn't...,im trying to teach maggie that nature doesnt e...,24.0
4,4,10374,35,35,"Lisa Simpson: It's like an ox, only it has a h...",254000,True,9,5.0,Lisa Simpson,Simpson Home,"It's like an ox, only it has a hump and a dewl...",its like an ox only it has a hump and a dewlap...,18.0


In [24]:
phrases = df['normalized_text'].tolist()
phrases[:10]

['maggie look whats that',
 'lee-mur lee-mur',
 'zee-boo zee-boo',
 'im trying to teach maggie that nature doesnt end with the barnyard i want her to have all the advantages that i didnt have',
 'its like an ox only it has a hump and a dewlap hump and dew-lap hump and dew-lap',
 'you know his blood type how romantic',
 'oh yeah whats my shoe size',
 'ring',
 'yes dad',
 'ooh look maggie what is that do-dec-ah-edron dodecahedron']

In [25]:
text= [[c for c in ph] for ph in phrases if type(ph) is str]

In [26]:
text[0]

['m',
 'a',
 'g',
 'g',
 'i',
 'e',
 ' ',
 'l',
 'o',
 'o',
 'k',
 ' ',
 'w',
 'h',
 'a',
 't',
 's',
 ' ',
 't',
 'h',
 'a',
 't']

# Делаем массив с данными

In [27]:
import string

CHARS = set('abcdefghijklmnopqrstuvwxyz ')
INDEX_TO_CHAR = ['none']+[w for w in CHARS]
CHAR_TO_INDEX = {w:i for i,w in enumerate(INDEX_TO_CHAR)}

In [28]:
len(INDEX_TO_CHAR)

28

In [29]:
MAX_LEN = 50
X = torch.zeros((len(text),MAX_LEN),dtype=int)
for i in range(len(text)):
  for j,w in enumerate(text[i]):
    if j >= MAX_LEN:
      break
    X[i,j] = CHAR_TO_INDEX.get(w,CHAR_TO_INDEX['none'])

In [30]:
X[0]

tensor([26, 24,  7,  7, 18, 13, 16,  2,  3,  3, 10, 16,  5, 14, 24, 19, 21, 16,
        19, 14, 24, 19,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0])

# Смотрим на Embedding и RNN ячейку

In [31]:
embeddings = torch.nn.Embedding(len(INDEX_TO_CHAR),28)
t = embeddings(X[0:10])
t

tensor([[[ 0.2390,  0.5352, -1.2377,  ...,  0.2157, -0.0305, -0.6526],
         [-1.1336, -0.4071,  2.0627,  ..., -0.3991, -0.6193,  0.5068],
         [ 0.3720,  0.2448, -0.0882,  ...,  0.8209, -0.7915,  1.3063],
         ...,
         [ 0.3772,  1.5255,  1.6081,  ..., -1.0302, -0.9624, -1.0719],
         [ 0.3772,  1.5255,  1.6081,  ..., -1.0302, -0.9624, -1.0719],
         [ 0.3772,  1.5255,  1.6081,  ..., -1.0302, -0.9624, -1.0719]],

        [[ 0.7180,  1.6823,  1.6155,  ...,  0.0212,  0.6062, -1.9738],
         [-0.1045, -0.3390,  1.7187,  ...,  1.3703,  0.9509, -0.0304],
         [-0.1045, -0.3390,  1.7187,  ...,  1.3703,  0.9509, -0.0304],
         ...,
         [ 0.3772,  1.5255,  1.6081,  ..., -1.0302, -0.9624, -1.0719],
         [ 0.3772,  1.5255,  1.6081,  ..., -1.0302, -0.9624, -1.0719],
         [ 0.3772,  1.5255,  1.6081,  ..., -1.0302, -0.9624, -1.0719]],

        [[ 0.9831, -0.5160, -0.5166,  ..., -1.1536, -1.0411, -0.9270],
         [-0.1045, -0.3390,  1.7187,  ...,  1

In [32]:
t.shape, X[0:10].shape

(torch.Size([10, 50, 28]), torch.Size([10, 50]))

In [33]:
rnn = torch.nn.RNN(28,128,batch_first=True) # 128 - Размер скрытого состояние
o, s = rnn(t)
o.shape, s.shape

(torch.Size([10, 50, 128]), torch.Size([1, 10, 128]))

In [34]:
o, s2 = rnn(t,s)
o.shape, s2.shape

(torch.Size([10, 50, 128]), torch.Size([1, 10, 128]))

# Метод предсказания следуюшего симбола

In [35]:
torch.nn.RNN(28,128,batch_first=True)

RNN(28, 128, batch_first=True)

In [46]:
class Network(torch.nn.Module):
  def __init__(self):
    super(Network,self).__init__()
    self.embed = torch.nn.Embedding(len(CHAR_TO_INDEX),28)
    self.rnn = torch.nn.RNN(28,128,batch_first=True,num_layers=10)
    self.linear = torch.nn.Linear(128,len(INDEX_TO_CHAR))
  def forward(self,sentences,state=None):
    embed = self.embed(sentences)
    o, s = self.rnn(embed)


    out = self.linear(o)
    return out

In [47]:
model = Network().cuda()

In [48]:
criterion = torch.nn.CrossEntropyLoss()

In [49]:
optimizer = torch.optim.SGD(model.parameters(),lr=0.05)

In [50]:
def generate_sentence(prn=False,gen_count =MAX_LEN):
  sentence = ['h','e','l','l','o']
  x = torch.zeros((1,len(sentence)),dtype=int).cuda()

  for j,w in enumerate(sentence):
    if j >=MAX_LEN:
      break
    x[0,j] = CHAR_TO_INDEX.get(w,CHAR_TO_INDEX['none'] )
 # print(x)
  for i in range(gen_count):
  #  print(i)
    o = model(x)
    w = torch.argmax(o[-1,-1,:],keepdim=True)
    if prn:
      print(x,w)
    x = torch.cat([x,w.unsqueeze(0)],axis=1)
    ww = INDEX_TO_CHAR[w]

    if ww == 'none':
      break
    sentence.append(ww) 
  return ''.join(sentence)


      

In [51]:
generate_sentence(True,5)

tensor([[14, 13,  2,  2,  3]], device='cuda:0') tensor([25], device='cuda:0')
tensor([[14, 13,  2,  2,  3, 25]], device='cuda:0') tensor([25], device='cuda:0')
tensor([[14, 13,  2,  2,  3, 25, 25]], device='cuda:0') tensor([25], device='cuda:0')
tensor([[14, 13,  2,  2,  3, 25, 25, 25]], device='cuda:0') tensor([25], device='cuda:0')
tensor([[14, 13,  2,  2,  3, 25, 25, 25, 25]], device='cuda:0') tensor([25], device='cuda:0')


'hellorrrrr'

In [52]:
for ep in range(200):
    start = time.time()
    train_loss = 0.
    train_passed = 0

    for i in range(int(len(X) / 100)):
        batch = X[i * 100:(i + 1) * 100].cuda()
        X_batch = batch[:, :-1]
        Y_batch = batch[:, 1:].flatten()

        optimizer.zero_grad()
        answers = model.forward(X_batch)
        answers = answers.view(-1, len(INDEX_TO_CHAR))
        loss = criterion(answers, Y_batch)
        train_loss += loss.item()

        loss.backward()
        optimizer.step()
        train_passed += 1
    if ep % 5 ==0:
      print(generate_sentence())
      print("Epoch {}. Time: {:.3f}, Train loss: {:.3f}".format(ep, time.time() - start, train_loss / train_passed))
    

hello                                                  
Epoch 0. Time: 0.329, Train loss: 2.295
hello to to to to to to to to to to to to to to to to t
Epoch 5. Time: 0.315, Train loss: 1.830
hello a to to to to to to to to to to to to to to to to
Epoch 10. Time: 0.316, Train loss: 1.753
hello to to to to to to to to to to to to to to to to t
Epoch 15. Time: 0.319, Train loss: 1.672
helloe to to to to to to to to to to to to to to to to 
Epoch 20. Time: 0.315, Train loss: 1.615
helloe to to to to to to to to to to to to to to to to 
Epoch 25. Time: 0.326, Train loss: 1.579
helloe to to to to to to to to to to to to to to to to 
Epoch 30. Time: 0.324, Train loss: 1.553
helloe to to to to to to to to to to to to to to to to 
Epoch 35. Time: 0.317, Train loss: 1.529
hello to to to to to to to to to to to to to to to to t
Epoch 40. Time: 0.347, Train loss: 1.508
hello the the the the the the the the the the the the t
Epoch 45. Time: 0.325, Train loss: 1.485
hello the the the the the the th

In [ ]:
X

In [30]:
Y_batch.shape

torch.Size([4900])

In [33]:
answers.shape

torch.Size([4900, 28])